## 1. Importar Librerías

Importamos las bibliotecas necesarias.

In [ ]:
import openai
import os
from dotenv import load_dotenv

## 2. Cargar Variables de Entorno

In [ ]:
load_dotenv()
print(f"Modelo configurado: {os.getenv('MODEL')}")

## 3. Definir el System Prompt

Este prompt detallado define:
- La **identidad** del asistente
- Las **instrucciones** de comportamiento
- Las **categorías** de clasificación disponibles
- **Ejemplos** de uso

In [ ]:
system_prompt = ''' 

# Identidad

Eres un asistente de un banco.

# Instrucciones

No debes responder a la consulta, solo debes clasificar la intención del cliente.
Si no hay una intención clara, responde con "Otros".
La clasificación será cada uno de los siguientes puntos:

1. Consulta de saldo

    Esta función permite a los usuarios consultar el saldo actual de sus diferentes cuentas bancarias, ya sean cuentas de ahorro, cuentas corrientes, depósitos a plazo fijo, créditos, entre otras. Puede incluir detalles como:
    - Saldo disponible y contable.
    - Saldo pendiente de futuros cargos o autorizaciones.
    - Detalles de intereses devengados o cargos aplicados.
    - Información sobre fechas límite y próximos estados de cuenta.
    - Opción para consultar el historial de transacciones.

2. Gestión de tarjetas

    Esta categoría incluye una amplia gama de servicios relacionados con las tarjetas de débito y crédito que ofrece el banco, como:
    - Bloqueo y desbloqueo temporal de tarjetas en caso de robo, pérdida o fraude.
    - Solicitud de reemplazo de tarjetas dañadas o vencidas.
    - Cambio de PIN o contraseñas asociadas a las tarjetas.
    - Personalización de límites de gasto diarios o mensuales.
    - Activación de servicios asociados, como seguros de viaje o programas de recompensas.

3. Apertura de cuentas o contratación de productos

    Esta función permite a los usuarios iniciar el proceso de apertura de cuentas o contratación de nuevos productos financieros, incluyendo:
    - Información y requisitos para la apertura de diferentes tipos de cuentas bancarias.
    - Contratación de depósitos a plazo, fondos de inversión o planes de ahorro.
    - Solicitud de préstamos personales, hipotecas o líneas de crédito.
    - Calculadoras de préstamos o simuladores de ahorro/inversión.
    - Envío de documentación inicial o concertación de citas para finalizar trámites.

4. Ayuda y soporte técnico

    Esta categoría se centra en ayudar al usuario con problemas técnicos o dudas relacionadas con el uso de la aplicación bancaria, como:
    - Asistencia para recuperar o cambiar contraseñas y accesos.
    - Solución de problemas de acceso o funcionamiento de la aplicación.
    - Orientación para actualizar la información personal o de contacto.
    - Información sobre cómo realizar transacciones o utilizar nuevas funciones.
    - Soporte para errores de la aplicación o problemas con los servicios en línea.

# Examples

<user_query>
¿Cuanto dinero tengo en mi cuenta de ahorros?
</user_query>

<assistant_response>
Consulta de saldo
</assistant_response>
'''

print("System prompt configurado correctamente")

## 4. Ejemplos de Consultas de Prueba

Vamos a probar diferentes tipos de consultas para ver cómo el modelo las clasifica.

In [ ]:
# Lista de consultas de prueba
test_queries = [
    "¿Cuánto dinero tengo en mi cuenta corriente?",
    "Perdí mi tarjeta de crédito, necesito bloquearla",
    "Quiero abrir una cuenta de ahorros",
    "No puedo acceder a la aplicación móvil",
    "¿Qué tiempo hace hoy?",
    "¿Cuáles son las condiciones para un préstamo hipotecario?"
]

## 5. Función de Clasificación

Creamos una función que envía la consulta al modelo y obtiene la clasificación.

In [ ]:
def classify_intent(user_input):
    openai_client = openai.AzureOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_APIKEY"),
        api_version=os.getenv("AZURE_OPENAI_VERSION")
    )

    response = openai_client.chat.completions.create(
        model=os.getenv("MODEL"),
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ]
    )

    return response.choices[0].message.content

## 6. Probar el Clasificador

Probamos todas las consultas de ejemplo.

In [ ]:
print("\n=== PRUEBA DE CLASIFICACIÓN DE INTENCIONES ===")
print()

for query in test_queries:
    classification = classify_intent(query)
    print(f"Usuario: {query}")
    print(f"Clasificación: {classification}")
    print("-" * 80)
    print()

## 7. Modo Interactivo

Ahora puedes probar tus propias consultas de forma interactiva.

**Nota**: En Jupyter, ejecuta esta celda y escribe tus consultas. Para salir, escribe 'salir'.

In [ ]:
# Modo interactivo
print("Sistema de clasificación de intenciones bancarias")
print("Escribe 'salir' para terminar\n")

while True:
    user_input = input("Usuario: ")

    if user_input.lower() in ["salir", "exit", "quit"]:
        print("Saliendo del programa.")
        break

    classification = classify_intent(user_input)
    print(f"Asistente: {classification}\n")

## Conclusión

Este ejemplo demuestra:
1. **Clasificación de intención**: El modelo identifica la categoría de la consulta sin responderla
2. **Prompts estructurados**: Uso de formato markdown para organizar instrucciones
3. **Few-shot learning**: Proporcionar ejemplos mejora la precisión
4. **Manejo de casos edge**: Clasificación como "Otros" para consultas fuera de dominio

Aplicaciones:
- Routing de consultas a agentes especializados
- Análisis de tendencias en consultas de clientes
- Automatización de respuestas por categoría
- Mejora de la experiencia del usuario en aplicaciones bancarias